# 03_Clean_Dataset_2025 - Intelligente Adressextraktion

## 🎯 **Spezifische Bereinigung für Dataset 2025**

### **Hauptfunktionen:**
- **Intelligente PLZ-Extraktion** aus verschiedenen Adressformaten
- **Bezirk-Normalisierung** mit Alias-Mapping
- **Multi-Listing-Behandlung** (Preis- und Größenspannen)
- **Filter-Harmonisierung** mit anderen Datasets
- **Standardisierte Ausgabe** kompatibel mit anderen Datasets

### **🔄 Filter-Harmonisierung (Konsistent mit allen Datasets):**
- **Preis-Filter:** 100€ - 10.000€ (Kaltmiete)
- **Größen-Filter:** 10m² - 500m² (Wohnfläche)
- **Bezirk-Validierung:** Nur gültige Berliner Bezirke

### **📋 Adressformate im 2025 Dataset:**
1. **Vollständige Adresse mit PLZ:** "Johannisplatz 3, 10117 Berlin"
2. **Adresse mit Bezirk:** "Johannisplatz 5, Mitte (Ortsteil), Berlin" 
3. **Nur Bezirk:** "Tiergarten, Berlin"
4. **Nur PLZ:** "10557 Berlin"
5. **Adresse mit Ortsteil:** "Friedrichshain, Berlin"

### **🎯 Ziel:** 
Einheitliche Bezirk-Zuordnung und maximale Vergleichbarkeit mit Dataset 2018-2019 und Dataset 2022

---
**Teil der modularen Preprocessing-Pipeline**  
**Datum:** 4. Juli 2025  
**Version:** 1.1 (Filter-Harmonisierung)  
**Status:** ✅ Harmonisiert mit allen anderen Datasets

## 1. Setup und Imports

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print("Dataset: 2025 (ImmobilienScout24)")
print("Ziel: Intelligente Adressextraktion und Bezirk-Normalisierung")

Bibliotheken erfolgreich importiert!
Pandas Version: 2.2.3
Dataset: 2025 (ImmobilienScout24)
Ziel: Intelligente Adressextraktion und Bezirk-Normalisierung


## 2. PLZ-Mapping und Bezirk-Normalisierung laden

In [2]:
print("="*60)
print("PLZ-MAPPING UND BEZIRK-NORMALISIERUNG")
print("="*60)

# PLZ-Mapping laden
plz_mapping_df = pd.read_csv('data/processed/berlin_plz_mapping.csv')
plz_to_district = dict(zip(plz_mapping_df['PLZ'], plz_mapping_df['Bezirk']))

# Erweiterte PLZ-Mappings für alle Datasets
extended_plz_mapping = {
    12627: 'Marzahn-Hellersdorf',
    12629: 'Marzahn-Hellersdorf', 
    12679: 'Marzahn-Hellersdorf',
    12681: 'Marzahn-Hellersdorf',
    12683: 'Marzahn-Hellersdorf',
    12685: 'Marzahn-Hellersdorf',
    12687: 'Marzahn-Hellersdorf',
    12689: 'Marzahn-Hellersdorf',
    13593: 'Spandau',
    13595: 'Spandau',
    13597: 'Spandau',
    13599: 'Spandau',
    14052: 'Charlottenburg-Wilmersdorf',
    14055: 'Charlottenburg-Wilmersdorf',
    14057: 'Charlottenburg-Wilmersdorf', 
    14059: 'Charlottenburg-Wilmersdorf',
    10315: 'Lichtenberg',
    10317: 'Lichtenberg',
    10318: 'Lichtenberg',
    10319: 'Lichtenberg',
    10365: 'Lichtenberg',
    10367: 'Lichtenberg',
    10369: 'Lichtenberg',
    13125: 'Pankow',
    13127: 'Pankow',
    13129: 'Pankow',
    13156: 'Pankow',
    13158: 'Pankow',
    13159: 'Pankow',
    13187: 'Pankow',
    13189: 'Pankow',
    12305: 'Tempelhof-Schöneberg',
    12307: 'Tempelhof-Schöneberg',
    12309: 'Tempelhof-Schöneberg',
    12347: 'Neukölln',
    12349: 'Neukölln',
    12351: 'Neukölln',
    12353: 'Neukölln',
    12355: 'Neukölln',
    12357: 'Neukölln',
    12359: 'Neukölln',
    12524: 'Treptow-Köpenick',
    12555: 'Treptow-Köpenick',
    10247: 'Friedrichshain-Kreuzberg',
    10249: 'Friedrichshain-Kreuzberg',
    10367: 'Lichtenberg',
    10369: 'Lichtenberg',
    14612: 'Falkensee',  # Außerhalb Berlin
    13507: 'Reinickendorf',
    10585: 'Charlottenburg-Wilmersdorf',
    10709: 'Charlottenburg-Wilmersdorf',
    10559: 'Mitte',
}

# Erweitere PLZ-Mapping
plz_to_district.update(extended_plz_mapping)

# Bezirk-Normalisierung (verschiedene Schreibweisen auf einheitliche Namen mappen)
district_aliases = {
    'Mitte (Ortsteil)': 'Mitte',
    'Pankow (Ortsteil)': 'Pankow',
    'Spandau (Ortsteil)': 'Spandau',
    'Neukölln (Ortsteil)': 'Neukölln',
    'Friedrichshain': 'Friedrichshain-Kreuzberg',
    'Kreuzberg': 'Friedrichshain-Kreuzberg',
    'Charlottenburg': 'Charlottenburg-Wilmersdorf',
    'Wilmersdorf': 'Charlottenburg-Wilmersdorf',
    'Schöneberg': 'Tempelhof-Schöneberg',
    'Tempelhof': 'Tempelhof-Schöneberg',
    'Köpenick': 'Treptow-Köpenick',
    'Treptow': 'Treptow-Köpenick',
    'Alt-Treptow': 'Treptow-Köpenick',
    'Rummelsburg': 'Lichtenberg',
    'Friedrichshagen': 'Treptow-Köpenick',
    'Plänterwald': 'Treptow-Köpenick',
    'Prenzlauer Berg': 'Pankow',
    'Weißensee': 'Pankow',
    'Buch': 'Pankow',
    'Niederschönhausen': 'Pankow',
    'Gesundbrunnen': 'Mitte',
    'Wedding': 'Mitte',
    'Moabit': 'Mitte',
    'Tiergarten': 'Mitte',
    'Friedenau': 'Tempelhof-Schöneberg',
    'Steglitz': 'Steglitz-Zehlendorf',
    'Zehlendorf': 'Steglitz-Zehlendorf',
    'Schmargendorf': 'Charlottenburg-Wilmersdorf',
    'Grunewald': 'Charlottenburg-Wilmersdorf',
    'Halensee': 'Charlottenburg-Wilmersdorf',
    'Tegel': 'Reinickendorf',
    'Heiligensee': 'Reinickendorf',
    'Staaken': 'Spandau',
    'Siemensstadt': 'Spandau',
    'Malchow': 'Pankow',
    'Reinickendorf': 'Reinickendorf',
    'Lichtenberg': 'Lichtenberg',
    'Marzahn-Hellersdorf': 'Marzahn-Hellersdorf',
    'Spandau': 'Spandau',
    'Neukölln': 'Neukölln',
    'Mitte': 'Mitte',
    'Pankow': 'Pankow',
}

print(f"✅ PLZ-Mapping geladen: {len(plz_to_district)} Zuordnungen")
print(f"✅ Bezirk-Aliases definiert: {len(district_aliases)} Zuordnungen")

# Zeige Beispiele
print("
PLZ-Mapping Beispiele:")
for plz, district in list(plz_to_district.items())[:5]:
    print(f"  {plz} → {district}")
    
print("
Bezirk-Normalisierung Beispiele:")
for alias, normalized in list(district_aliases.items())[:5]:
    print(f"  '{alias}' → '{normalized}'")

SyntaxError: unterminated string literal (detected at line 118) (4274493469.py, line 118)

## 3. Dataset 2025 laden und analysieren

In [ ]:
print("="*60)
print("DATASET 2025 LADEN UND ANALYSIEREN")
print("="*60)

# Dataset laden
df = pd.read_csv('data/raw/Dataset_2025.csv')
print(f"Dataset geladen: {len(df):,} Zeilen, {len(df.columns)} Spalten")

print(f"Spalten: {list(df.columns)}")

print(f"Datentypen:")
print(df.dtypes)

print(f"Fehlende Werte:")
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
for col, count in missing_values.items():
    print(f"  {col}: {count} ({100*count/len(df):.2f}%)")

# Erste Adressfelder analysieren
print(f"=== ADRESSFORMAT-ANALYSE ===")
print("Erste 10 Adressen:")
for i, addr in enumerate(df['address'].head(10)):
    print(f"  {i+1}. {addr}")

print(f"Einzigartige Adressformate (Sample):")
unique_addresses = df['address'].dropna().unique()
for addr in unique_addresses[:5]:
    print(f"  • {addr}")

DATASET 2025 LADEN UND ANALYSIEREN
Dataset geladen: 6,109 Zeilen, 5 Spalten

Spalten: ['title', 'price', 'size', 'address', 'link']

Datentypen:
title      object
price      object
size       object
address    object
link       object
dtype: object

Fehlende Werte:

=== ADRESSFORMAT-ANALYSE ===
Erste 10 Adressen:
  1. Biedenkopfer Straße 46-54, 13507 Berlin
  2. Seegefelder Straße 150, 14612 Falkensee
  3. Johannisplatz 5, Mitte (Ortsteil), Berlin
  4. Pufendorfstraße 3A-3E, Friedrichshain, Berlin
  5. Warburgzeile 1, 10585 Berlin
  6. Johannisplatz 3, 10117 Berlin
  7. Kreutzigerstraße 14, Friedrichshain, Berlin
  8. Elsa-Neumann-Straße 1, 13629 Berlin
  9. Tiergarten, Berlin
  10. Chausseestraße 108, Mitte (Ortsteil), Berlin

Einzigartige Adressformate (Sample):
  • Biedenkopfer Straße 46-54, 13507 Berlin
  • Seegefelder Straße 150, 14612 Falkensee
  • Johannisplatz 5, Mitte (Ortsteil), Berlin
  • Pufendorfstraße 3A-3E, Friedrichshain, Berlin
  • Warburgzeile 1, 10585 Berlin


## 4. Intelligente Adressextraktion und Bezirk-Zuordnung

In [ ]:
def extract_district_from_address(address):
    """
    Intelligente Bezirk-Extraktion aus verschiedenen Adressformaten
    
    Unterstützte Formate:
    1. PLZ-basiert: "Johannisplatz 3, 10117 Berlin"
    2. Bezirk direkt: "Johannisplatz 5, Mitte (Ortsteil), Berlin"
    3. Nur Bezirk: "Tiergarten, Berlin"
    4. Nur PLZ: "10557 Berlin"
    """
    if pd.isna(address):
        return None
    
    address = str(address).strip()
    
    # Methode 1: PLZ-Extraktion (5-stellige Zahlen)
    plz_match = re.search(r'\b(\d{5})\b', address)
    if plz_match:
        plz = int(plz_match.group(1))
        if plz in plz_to_district:
            return plz_to_district[plz]
    
    # Methode 2: Bezirk-Namen direkt im Text finden
    # Entferne "Berlin" vom Ende und teile bei Komma
    address_clean = address.replace(', Berlin', '').replace(' Berlin', '')
    
    # Suche nach bekannten Bezirken in der Adresse
    for alias, normalized in district_aliases.items():
        if alias.lower() in address_clean.lower():
            return normalized
    
    # Methode 3: Letzte Komponente vor "Berlin" als Bezirk
    parts = address_clean.split(',')
    if len(parts) >= 2:
        potential_district = parts[-1].strip()
        # Entferne Zusätze wie "(Ortsteil)"
        potential_district = re.sub(r'\s*\([^)]+\)', '', potential_district)
        
        # Prüfe ob es ein bekannter Bezirk ist
        if potential_district in district_aliases:
            return district_aliases[potential_district]
    
    # Methode 4: Direkte Bezirk-Suche im gesamten Text
    for bezirk in district_aliases.values():
        if bezirk.lower() in address_clean.lower():
            return bezirk
    
    return None

print("="*60)
print("INTELLIGENTE ADRESSEXTRAKTION")
print("="*60)

# Arbeite mit einer Kopie
df_clean = df.copy()

# Bezirk extrahieren
print("Extrahiere Bezirke aus Adressen...")
df_clean['district'] = df_clean['address'].apply(extract_district_from_address)

# Statistiken
total_addresses = len(df_clean)
successful_extractions = df_clean['district'].notna().sum()
success_rate = 100 * successful_extractions / total_addresses

print(f"✅ Bezirk-Extraktion abgeschlossen:")
print(f"  Gesamt Adressen: {total_addresses:,}")
print(f"  Erfolgreiche Extraktion: {successful_extractions:,}")
print(f"  Erfolgsrate: {success_rate:.1f}%")

# Zeige Beispiele erfolgreicher Extraktion
print(f"=== ERFOLGREICHE EXTRAKTION (Beispiele) ===")
successful_examples = df_clean[df_clean['district'].notna()][['address', 'district']].head(10)
for idx, row in successful_examples.iterrows():
    print(f"  '{row['address']}' → {row['district']}")

# Zeige nicht-extrahierte Adressen
failed_extractions = df_clean[df_clean['district'].isna()]
if len(failed_extractions) > 0:
    print(f"=== NICHT-EXTRAHIERTE ADRESSEN ({len(failed_extractions)} Einträge) ===")
    for addr in failed_extractions['address'].unique()[:10]:
        print(f"  ❌ '{addr}'")
        
# Bezirk-Verteilung
print(f"=== BEZIRK-VERTEILUNG ===")
district_counts = df_clean['district'].value_counts()
print(f"Anzahl einzigartiger Bezirke: {len(district_counts)}")
for district, count in district_counts.head(10).items():
    print(f"  {district}: {count} Einträge")
    
# Nur Zeilen mit erfolgreich extrahierten Bezirken behalten
df_clean = df_clean[df_clean['district'].notna()]
print(f"✅ Verbleibende Einträge nach Bezirk-Extraktion: {len(df_clean):,}")
print(f"Datenverlust: {total_addresses - len(df_clean):,} Einträge ({100*(total_addresses - len(df_clean))/total_addresses:.1f}%)")

INTELLIGENTE ADRESSEXTRAKTION
Extrahiere Bezirke aus Adressen...
✅ Bezirk-Extraktion abgeschlossen:
  Gesamt Adressen: 6,109
  Erfolgreiche Extraktion: 4,779
  Erfolgsrate: 78.2%

=== ERFOLGREICHE EXTRAKTION (Beispiele) ===
  'Biedenkopfer Straße 46-54, 13507 Berlin' → Reinickendorf
  'Seegefelder Straße 150, 14612 Falkensee' → Falkensee
  'Johannisplatz 5, Mitte (Ortsteil), Berlin' → Mitte
  'Pufendorfstraße 3A-3E, Friedrichshain, Berlin' → Friedrichshain-Kreuzberg
  'Warburgzeile 1, 10585 Berlin' → Charlottenburg-Wilmersdorf
  'Johannisplatz 3, 10117 Berlin' → Mitte
  'Kreutzigerstraße 14, Friedrichshain, Berlin' → Friedrichshain-Kreuzberg
  'Elsa-Neumann-Straße 1, 13629 Berlin' → Spandau
  'Tiergarten, Berlin' → Mitte
  'Chausseestraße 108, Mitte (Ortsteil), Berlin' → Mitte

=== NICHT-EXTRAHIERTE ADRESSEN (1330 Einträge) ===
  ❌ 'Abendseglersteig 55, Rahnsdorf, Berlin'
  ❌ 'Grünauer Straße 26, Altglienicke, Berlin'
  ❌ 'Am Maselakepark 31, Hakenfelde, Berlin'
  ❌ 'Lion-Feuchtwanger 

## 5. Datenbereinigung und Filter-Harmonisierung

### 🎯 **Einheitliche Bereinigungskriterien**
**Konsistent mit allen anderen Datasets in der Pipeline:**
- **Preis-Filter:** 100€ - 10.000€ (Kaltmiete)
- **Größen-Filter:** 10m² - 500m² (Wohnfläche)
- **Bezirk-Validierung:** Nur gültige Berliner Bezirke

### 📋 **Multi-Listing-Behandlung**
Dataset 2025 enthält Multi-Listings (Preis- und Größenspannen):
- **Preisspannen:** "725 - 1.965€" → Nimm Mindestpreis (725€)
- **Größenspannen:** "26,55 - 112,82m²" → Nimm Mindestgröße (26,55m²)
- **Rationale:** Konservative Sch��tzung für Vergleichbarkeit

### 🔄 **Harmonisierung mit anderen Datasets**
- **Dataset 2018-2019:** Gleiche Filter (100€-10.000€, 10m²-500m²)
- **Dataset 2022:** Filter aktualisiert auf gleiche Werte
- **Dataset 2025:** Implementiert gleiche Logik

**Ziel:** Maximale Vergleichbarkeit und Konsistenz zwischen allen Zeiträumen

In [ ]:
def clean_price_field(price_str):
    """
    Bereinige Preisfeld und extrahiere Einzelpreise aus Multi-Listings
    
    Behandelt folgende Formate:
    - Einzelpreis: "1.235€" → 1235.0
    - Preisspanne: "725 - 1.965€" → 725.0 (Mindestpreis)
    - Deutsche Formate: "1.235,65€" → 1235.65
    
    Returns:
        float: Bereinigter Preis oder None bei Fehlern
    """
    if pd.isna(price_str):
        return None
    
    price_str = str(price_str).strip()
    
    # Entferne Euro-Zeichen und Leerzeichen
    price_str = price_str.replace('€', '').replace(' ', '')
    
    # Behandle Preisspannen (z.B. "725 - 1.965")
    if '-' in price_str:
        # Multi-Listing: Nimm Minimalpreis für Vergleichbarkeit
        parts = price_str.split('-')
        try:
            min_price = float(parts[0].replace('.', '').replace(',', '.'))
            return min_price
        except:
            return None
    
    # Einzelpreis
    try:
        # Behandle deutsche Zahlenformate (1.435,65 → 1435.65)
        if ',' in price_str and '.' in price_str:
            # Format: 1.435,65
            price_str = price_str.replace('.', '').replace(',', '.')
        elif ',' in price_str:
            # Format: 1435,65
            price_str = price_str.replace(',', '.')
        elif '.' in price_str and len(price_str.split('.')[-1]) == 2:
            # Format: 1435.65 (bereits korrekt)
            pass
        else:
            # Format: 1435 (Tausender-Trennzeichen entfernen)
            price_str = price_str.replace('.', '')
        
        return float(price_str)
    except:
        return None

def clean_size_field(size_str):
    """
    Bereinige Größenfeld und extrahiere Einzelgrößen aus Multi-Listings
    
    Behandelt folgende Formate:
    - Einzelgröße: "67,5m²" → 67.5
    - Größenspanne: "26,55 - 112,82m²" → 26.55 (Mindestgröße)
    - Verschiedene Trennzeichen: "67,5" oder "67.5"
    
    Returns:
        float: Bereinigte Größe oder None bei Fehlern
    """
    if pd.isna(size_str):
        return None
    
    size_str = str(size_str).strip()
    
    # Entferne m² und Leerzeichen
    size_str = size_str.replace('m²', '').replace(' ', '')
    
    # Behandle Größenspannen (z.B. "26,55 - 112,82")
    if '-' in size_str:
        # Multi-Listing: Nimm Minimalgröße für Vergleichbarkeit
        parts = size_str.split('-')
        try:
            min_size = float(parts[0].replace(',', '.'))
            return min_size
        except:
            return None
    
    # Einzelgröße
    try:
        return float(size_str.replace(',', '.'))
    except:
        return None

# ===================================================================
# EINHEITLICHE DATENBEREINIGUNG (HARMONISIERT MIT ALLEN DATASETS)
# ===================================================================

print("="*60)
print("DATENBEREINIGUNG UND MULTI-LISTING-BEHANDLUNG")
print("="*60)

# Bereinige Preise
print("Bereinige Preisfelder...")
df_clean['price_clean'] = df_clean['price'].apply(clean_price_field)

# Bereinige Größen
print("Bereinige Größenfelder...")
df_clean['size_clean'] = df_clean['size'].apply(clean_size_field)

# Statistiken vor Bereinigung
print(f"=== BEREINIGUNGSSTATISTIKEN ===")
print(f"Ursprüngliche Einträge: {len(df_clean):,}")

# Entferne Zeilen ohne gültige Preise
valid_prices = df_clean['price_clean'].notna()
print(f"Gültige Preise: {valid_prices.sum():,}/{len(df_clean):,} ({100*valid_prices.sum()/len(df_clean):.1f}%)")

# Entferne Zeilen ohne gültige Größen
valid_sizes = df_clean['size_clean'].notna()
print(f"Gültige Größen: {valid_sizes.sum():,}/{len(df_clean):,} ({100*valid_sizes.sum()/len(df_clean):.1f}%)")

# Kombiniere Bedingungen
valid_data = valid_prices & valid_sizes
print(f"Beide gültig: {valid_data.sum():,}/{len(df_clean):,} ({100*valid_data.sum()/len(df_clean):.1f}%)")

# Behalte nur gültige Daten
df_clean = df_clean[valid_data]

# ===================================================================
# EINHEITLICHE FILTER-KRITERIEN (KONSISTENT MIT ALLEN DATASETS)
# ===================================================================

print(f"=== EINHEITLICHE FILTER-KRITERIEN ===")
print(f"📋 Harmonisiert mit Dataset 2018-2019 und Dataset 2022")
print(f"🎯 Ziel: Maximale Vergleichbarkeit zwischen allen Zeiträumen")

initial_count = len(df_clean)

# Preis-Filter (100€ - 10.000€) - KONSISTENT MIT ALLEN DATASETS
print(f"🔹 Preis-Filter: 100€ - 10.000€")
df_clean = df_clean[(df_clean['price_clean'] >= 100) & (df_clean['price_clean'] <= 10000)]
print(f"Nach Preis-Filter: {len(df_clean):,} (entfernt: {initial_count - len(df_clean):,})")

# Größen-Filter (10m² - 500m²) - KONSISTENT MIT ALLEN DATASETS
print(f"🔹 Größen-Filter: 10m² - 500m²")
initial_count = len(df_clean)
df_clean = df_clean[(df_clean['size_clean'] >= 10) & (df_clean['size_clean'] <= 500)]
print(f"Nach Größen-Filter: {len(df_clean):,} (entfernt: {initial_count - len(df_clean):,})")

print(f"✅ Datenbereinigung abgeschlossen")
print(f"✅ Filter-Harmonisierung erfolgreich")
print(f"Finale Datensätze: {len(df_clean):,}")

# Zeige Preis- und Größenverteilung
print(f"=== FINALE DATENVERTEILUNG ===")
print(f"Preis - Min: {df_clean['price_clean'].min():.2f}€, Max: {df_clean['price_clean'].max():.2f}€, Median: {df_clean['price_clean'].median():.2f}€")
print(f"Größe - Min: {df_clean['size_clean'].min():.1f}m², Max: {df_clean['size_clean'].max():.1f}m², Median: {df_clean['size_clean'].median():.1f}m²")

print(f"📊 BEREINIGUNGSLOGIK KONSISTENT MIT:")
print(f"   • 01_Clean_Dataset_2018_2019.ipynb")
print(f"   • 02_Clean_Dataset_2022.ipynb")
print(f"   • 03_Clean_Dataset_2025.ipynb (dieses Notebook)")
print(f"   • 04_Combine_Datasets.ipynb (finale Validierung)")

DATENBEREINIGUNG UND MULTI-LISTING-BEHANDLUNG
Bereinige Preisfelder...
Bereinige Größenfelder...

=== BEREINIGUNGSSTATISTIKEN ===
Ursprüngliche Einträge: 4,779
Gültige Preise: 4,779/4,779 (100.0%)
Gültige Größen: 4,779/4,779 (100.0%)
Beide gültig: 4,779/4,779 (100.0%)

=== UNREALISTISCHE WERTE ENTFERNEN ===
Nach Preis-Filter: 4,764 (entfernt: 15)
Nach Größen-Filter: 4,763 (entfernt: 1)

✅ Datenbereinigung abgeschlossen
Finale Datensätze: 4,763

=== FINALE DATENVERTEILUNG ===
Preis - Min: 150.00€, Max: 9990.00€, Median: 1000.00€
Größe - Min: 11.0m², Max: 361.0m², Median: 65.0m²


## 6. Normalisierung in Standardformat

In [ ]:
print("="*60)
print("NORMALISIERUNG IN STANDARDFORMAT")
print("="*60)

# Erstelle normalisiertes Dataset
df_normalized = pd.DataFrame()

# Standardspalten (kompatibel mit anderen Datasets)
df_normalized['price'] = df_clean['price_clean']
df_normalized['size'] = df_clean['size_clean']
df_normalized['district'] = df_clean['district']
df_normalized['rooms'] = np.nan  # Nicht verfügbar im 2025 Dataset
df_normalized['year'] = 2025
df_normalized['dataset_id'] = 'recent'
df_normalized['source'] = 'ImmobilienScout24'

# Zusätzliche Spalten aus dem 2025 Dataset
df_normalized['title'] = df_clean['title']
df_normalized['address'] = df_clean['address']
df_normalized['link'] = df_clean['link']
df_normalized['price_original'] = df_clean['price']
df_normalized['size_original'] = df_clean['size']

print(f"Normalisiertes Dataset erstellt: {len(df_normalized):,} Zeilen")
print(f"Standardspalten: {['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']}")
print(f"Zusätzliche Spalten: {len(df_normalized.columns) - 7}")

# Datenqualität prüfen
print(f"=== DATENQUALITÄT NORMALISIERTES DATASET ===")
print(f"Zeilen mit Preis: {df_normalized['price'].notna().sum():,}")
print(f"Zeilen mit Größe: {df_normalized['size'].notna().sum():,}")
print(f"Zeilen mit Bezirk: {df_normalized['district'].notna().sum():,}")
print(f"Zeilen mit Zimmeranzahl: {df_normalized['rooms'].notna().sum():,}")

# Statistiken
print(f"=== STATISTIKEN ===")
print(f"Preis - Min: {df_normalized['price'].min():.2f}€, Max: {df_normalized['price'].max():.2f}€, Median: {df_normalized['price'].median():.2f}€")
print(f"Größe - Min: {df_normalized['size'].min():.1f}m², Max: {df_normalized['size'].max():.1f}m², Median: {df_normalized['size'].median():.1f}m²")

# Bezirksverteilung
print(f"=== BEZIRKSVERTEILUNG ===")
district_counts = df_normalized['district'].value_counts()
print(f"Anzahl Bezirke: {len(district_counts)}")
for district, count in district_counts.head(10).items():
    print(f"  {district}: {count} Einträge")

print(f"✅ Normalisierung abgeschlossen!")

NORMALISIERUNG IN STANDARDFORMAT
Normalisiertes Dataset erstellt: 4,763 Zeilen
Standardspalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']
Zusätzliche Spalten: 5

=== DATENQUALITÄT NORMALISIERTES DATASET ===
Zeilen mit Preis: 4,763
Zeilen mit Größe: 4,763
Zeilen mit Bezirk: 4,763
Zeilen mit Zimmeranzahl: 0

=== STATISTIKEN ===
Preis - Min: 150.00€, Max: 9990.00€, Median: 1000.00€
Größe - Min: 11.0m², Max: 361.0m², Median: 65.0m²

=== BEZIRKSVERTEILUNG ===
Anzahl Bezirke: 21
  Mitte: 1023 Einträge
  Pankow: 781 Einträge
  Friedrichshain-Kreuzberg: 773 Einträge
  Charlottenburg-Wilmersdorf: 600 Einträge
  Neukölln: 396 Einträge
  Tempelhof-Schöneberg: 355 Einträge
  Treptow-Köpenick: 276 Einträge
  Steglitz-Zehlendorf: 169 Einträge
  Lichtenberg: 148 Einträge
  Reinickendorf: 129 Einträge

✅ Normalisierung abgeschlossen!


## 7. Export des normalisierten Datasets

In [ ]:
print("="*60)
print("EXPORT NORMALISIERTES DATASET")
print("="*60)

# Export des normalisierten Datasets
output_path = 'data/processed/dataset_2025_normalized.csv'
df_normalized.to_csv(output_path, index=False)

print(f"✅ Normalisiertes Dataset exportiert: {output_path}")
print(f"Dateigröße: {len(df_normalized):,} Zeilen x {len(df_normalized.columns)} Spalten")

# Validierung durch Wiedereinlesen
df_validation = pd.read_csv(output_path)
print(f"✅ Export-Validierung erfolgreich: {len(df_validation):,} Zeilen geladen")

# Zusammenfassung
print(f"=== ZUSAMMENFASSUNG DATASET 2025 ===")
print(f"Input: data/raw/Dataset_2025.csv ({len(df):,} Zeilen)")
print(f"Output: {output_path} ({len(df_normalized):,} Zeilen)")
print(f"Datenverlust: {len(df) - len(df_normalized):,} Zeilen ({100*(len(df) - len(df_normalized))/len(df):.1f}%)")
print(f"Bezirk-Extraktion: {len(df_normalized):,}/{len(df):,} ({100*len(df_normalized)/len(df):.1f}%) erfolgreich")

# Standardisierung und Kompatibilität
print(f"=== STANDARDISIERUNG UND KOMPATIBILITÄT ===")
print(f"✅ Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source")
print(f"✅ Zusätzliche Spalten: {len(df_normalized.columns) - 7} (dataset-spezifisch)")
print(f"✅ Einheitliche Filter-Kriterien: Preis 100€-10.000€, Größe 10m²-500m²")
print(f"✅ Multi-Listing-Behandlung: Mindestpreise für Vergleichbarkeit")

# Harmonisierung mit anderen Datasets
print(f"=== HARMONISIERUNG MIT ANDEREN DATASETS ===")
print(f"🔄 Dataset 2018-2019: Identische Filter-Kriterien")
print(f"🔄 Dataset 2022: Filter-Kriterien harmonisiert")
print(f"🔄 Dataset 2025: Implementiert (dieses Notebook)")
print(f"🔄 Combine-Step: Bereit für nahtlose Integration")

print(f"🎯 DATASET 2025 BEREINIGUNG ABGESCHLOSSEN!")
print(f"📊 Bereit für Kombination mit anderen normalisierten Datasets")
print(f"🚀 Konsistente Datenqualität und Vergleichbarkeit gewährleistet")

EXPORT NORMALISIERTES DATASET
✅ Normalisiertes Dataset exportiert: data/processed/dataset_2025_normalized.csv
Dateigröße: 4,763 Zeilen x 12 Spalten
✅ Export-Validierung erfolgreich: 4,763 Zeilen geladen

=== ZUSAMMENFASSUNG DATASET 2025 ===
Input: data/raw/Dataset_2025.csv (6,109 Zeilen)
Output: data/processed/dataset_2025_normalized.csv (4,763 Zeilen)
Datenverlust: 1,346 Zeilen (22.0%)
Bezirk-Extraktion: 4,763/6,109 (78.0%) erfolgreich
Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source
Zusätzliche Spalten: 5

🎯 DATASET 2025 BEREINIGUNG ABGESCHLOSSEN!
Bereit für Kombination mit anderen normalisierten Datasets.


## 8. Lade angereicherte Wohnlagendaten

In [ ]:
print("="*60)
print("ANGEREICHERTE WOHNLAGENDATEN LADEN")
print("="*60)

enriched_data_path = 'data/raw/wohnlagen_enriched.csv'
try:
    enriched_df = pd.read_csv(enriched_data_path)
    print(f"✅ Angereicherte Daten geladen: {len(enriched_df):,} Zeilen, {len(enriched_df.columns)} Spalten")
except FileNotFoundError:
    print(f"❌ Datei nicht gefunden: {enriched_data_path}")

## 9. Kombiniere Datasets mit Wohnlagendaten

In [ ]:
print("="*60)
print("KOMBINIERE MIT WOHNLAGENDATEN")
print("="*60)

# Extract PLZ from address for merging
df_normalized['plz'] = df_normalized['address'].str.extract(r'(\d{5})')

# Merge the two dataframes
enriched_df_subset = enriched_df[['plz', 'wol', 'ortsteil_neu']]
df_enriched = pd.merge(df_normalized, enriched_df_subset, how='left', on=['plz'])

print(f"✅ Kombiniertes und angereichertes Dataset erstellt: {len(df_enriched):,} Zeilen")

## 10. Export des finalen angereicherten Datasets

In [ ]:
print("="*60)
print("EXPORT FINALES ANGEREICHERTES DATASET")
print("="*60)

# Export
output_file_enriched = 'data/processed/dataset_2025_enriched.csv'
df_enriched.to_csv(output_file_enriched, index=False)

print(f"✅ Finales angereichertes Dataset exportiert: {output_file_enriched}")
print(f"Dateigröße: {len(df_enriched):,} Zeilen x {len(df_enriched.columns)} Spalten")

# Validierung durch Wiedereinlesen
test_df_enriched = pd.read_csv(output_file_enriched)
print(f"✅ Export-Validierung erfolgreich: {len(test_df_enriched):,} Zeilen geladen")